In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy import stats
import pandas as pd
from math import *
import seaborn as sns
from mpl_toolkits.basemap import Basemap

In [2]:
data = pd.read_csv('NpdbPublicUseDataCsv/NPDB1710.CSV',delimiter=',',low_memory=False)

In [36]:
state_pop = pd.read_excel('nst-est2016-01.xlsx',header=3,skip_footer=5)

new_index_list = {}
for s_label in state_pop.index:
    try:
        if s_label[0]=='.':
            new_index_list.update({s_label: s_label[1:]})
        else:
            new_index_list.update({s_label: s_label})
    except:
        new_index_list.update({s_label: 'unknown'})

#print(new_index_list)
state_pop.rename(new_index_list, axis='index',inplace=True)
#state_pop

In [37]:
#data.hist(column='ORIGYEAR',bins=2018-1990,range=(1990,2018),alpha=0.5,by='WORKSTAT',figsize=(20,40),layout=(13,5))
#plt.show()

In [38]:
common_states = np.intersect1d(list(data["WORKSTAT"].unique()),list(state_pop["State Abb"]))

In [39]:
data["PAYMENT"].replace('[\$,]', '', regex=True, inplace=True)

In [ ]:
pay_count = None
for state in common_states:
    #print(state)
    arrays = [[state,state], ['pay_per_case', 'count_per_10000']]
    column_names = pd.MultiIndex.from_arrays(arrays).T
    mean_payment = [pd.to_numeric(data[(data["WORKSTAT"]==state) & (data["ORIGYEAR"]==i)]['PAYMENT']).mean() for i in range(1990,2018)]
    count_per_capita = [10000*len(data[(data["WORKSTAT"]==state) & (data["ORIGYEAR"]==i)])/state_pop[state_pop["State Abb"]==state][2016][0] for i in range(1990,2018)]
    stack = pd.DataFrame(np.stack((mean_payment,
                                   count_per_capita),axis=0).T,
                         columns=column_names,
                         index=range(1990,2018))
    #print(stack)
    if type(pay_count)==None:
        pay_count = stack
    else:
        pay_count = pd.concat([pay_count, stack], axis=1)

In [40]:
correlation_count = np.zeros((len(common_states),len(common_states)))

for state1 in range(len(common_states)):
    for state2 in range(len(common_states)):
        slope, intercept, rvalue, pvalue, stderr = scipy.stats.linregress(pay_count[common_states[state1],"count_per_10000"],pay_count[common_states[state2],"count_per_10000"])
        correlation_count[state1,state2] = rvalue


#plt.hist((correlation_count+1).flatten())
#plt.show()

In [82]:
from sklearn.cluster import DBSCAN
# Compute DBSCAN

db = DBSCAN(eps=0.56, min_samples=1,metric='precomputed').fit_predict(1/(correlation_count+1))
#core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
#core_samples_mask[db.core_sample_indices_] = True
#db

array([ 0,  1,  2,  1,  2,  2,  0,  3,  4,  2,  2,  5,  2,  6,  2,  7,  2,
        8,  6,  2,  9, 10,  2, 11,  2,  2,  2, 12,  4, 13, 14, 15, 16,  6,
        2,  2, 17,  6,  2, 18, 19,  1, 20,  2, 21, 22,  0, 23,  6, 24,  2,
       16])

In [83]:
delta_count = []
std_delta_count = []
for state in common_states:
    slope = np.array(pay_count[state]['count_per_10000'][2:])-np.array(pay_count[state]['count_per_10000'][1:-1])
    delta_count.append(np.mean(np.array(slope)))
    std_delta_count.append(np.std(np.array(slope)))
    #print(state, np.mean(np.array(slope)), np.std(np.array(slope)))

In [85]:
for i in range(len(np.unique(db))):
    #print("CLUSTER "+str(i))
    #data[data["WORKSTAT"].isin(list(common_states[np.where(db==i)]))].hist(column='ORIGYEAR',bins=2018-1990,range=(1990,2018),alpha=0.5,by='WORKSTAT',figsize=(20,40),layout=(7,3))
    #plt.show()
    slope = []
    for state in list(common_states[np.where(db==i)]):
    #    plt.plot(pay_count.index,pay_count[state]['count_per_10000'])
        #print(np.array(pay_count[state]['count_per_10000'].iloc[1:]))
        #print(pay_count[state]['count_per_10000'][:-1])
        slope.append(np.array(pay_count[state]['count_per_10000'][2:])-np.array(pay_count[state]['count_per_10000'][1:-1]))
    #plt.show()
    #print(slope)
    #print(np.mean(np.array(slope).flatten()),np.std(np.array(slope).flatten()))
    

In [46]:
from bokeh.io import show
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LogColorMapper,
    LabelSet,
    Label
)
from bokeh.palettes import Viridis6 as palette
from bokeh.plotting import figure

In [48]:
from bokeh.sampledata.us_states import data as states

del states["HI"]
del states["AK"]

palette.reverse()

In [49]:
states = {
    code: state for code, state in states.items()
}
#print(states)

In [50]:
state_xs = [states[code]["lons"] for code in states]
state_ys = [states[code]["lats"] for code in states]

In [51]:
state_names = [state['name'] for state in states.values()]

In [52]:
state_rates = []
for name in state_names:
    abbr = state_pop.loc[name]["State Abb"]
    state_rates.append(delta_count[np.where(common_states==abbr)[0][0]])

In [53]:
lat_inkm = 111.132 ## at around lat = 45degrees from the wiki latitude page
lon_inkm = 78.847

In [54]:
text_position_x = []
for statex in state_xs:
    text_position_x.append(float("%.5f" % round(max(statex)-(max(statex)-min(statex))/2,5)))
    
text_position_y = []
for statey in state_ys:
    text_position_y.append(float("%.5f" % round(max(statey)-(max(statey)-min(statey))/2,5)))

In [55]:
from bokeh.palettes import Spectral11,Category10

db_order = []
for name in state_names:
    abbr = state_pop.loc[name]["State Abb"]
    db_order.append(db[np.where(common_states==abbr)[0][0]])

In [56]:
db_spectral = []
for i in db_order:
    #print(i,len(np.where(db_order==i)[0]),i%11)
    if len(np.where(db_order==i)[0])==1:
        db_spectral.append(-1)
    if len(np.where(db_order==i)[0])>1:
        db_spectral.append(i)
        
#print(db_spectral)
db_spectral_arr = np.array(db_spectral)
#print(np.unique(db_spectral_arr[np.where(db_spectral_arr!=-1)[0]]))
ordered = 0
for label in np.unique(db_spectral_arr[np.where(db_spectral_arr!=-1)[0]]):
    db_spectral_arr[np.where(db_spectral_arr==label)[0]]=ordered
    ordered += 1

In [57]:
state_clusters = []
for i in range(len(db_spectral_arr)):
    if db_spectral_arr[i]==-1:
        state_clusters.append('black')
    else:
        state_clusters.append(Category10[10][db_spectral_arr[i]])

In [58]:
from bokeh.models import LogColorMapper, LogTicker, ColorBar
from bokeh.resources import CDN
from bokeh.embed import file_html

In [1]:
from bokeh.io import export_png

color_mapper = LogColorMapper(palette=palette, low=min(state_rates), high=max(state_rates))


source = ColumnDataSource(data=dict(
    x=state_xs,
    y=state_ys,
    name=state_names,
    rate=state_rates,
    clusters=state_clusters,
))

TOOLS = "pan,wheel_zoom,reset,hover,save"

p = figure(title="Change in the number of malpractice cases filed per 10,000 citizens per states, 1990-2018", 
    plot_width=int((max(max(state_xs))-min(min(state_xs)))*lon_inkm/4.5), 
    plot_height=int((max(max(state_ys))-min(min(state_ys)))*lat_inkm/4.5), tools=TOOLS,
    x_axis_location=None, y_axis_location=None
)
p.grid.grid_line_color = None

p.patches('x', 'y', source=source,
          fill_color={'field': 'rate', 'transform': color_mapper},
          fill_alpha=0.7, line_color="white", line_width=0.5)

color_bar = ColorBar(color_mapper=color_mapper, ticker=LogTicker(),
                     label_standoff=12, border_line_color=None, location=(0,0))

p.add_layout(color_bar, 'right')

source2 = ColumnDataSource(data=dict(
    x=text_position_x,
    y=text_position_y,
    name=state_names,
    rate=state_rates,
    clusters=state_clusters,
))

#labels = LabelSet(x='x', y='y', text='name', level='glyph',
#              x_offset=-15, y_offset=-1, source=source2,
#                  text_color='clusters',render_mode='canvas')

#p.add_layout(labels)

hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips ="""
    <font size="3">State: <strong>@name</strong> </font> <br>
    <font size="3">Change in the number of malpractice cases </font> <br>
    <font size="3">per year from 1990 to 2018: <strong>@rate per 10,000</strong> </font>
"""
from bokeh.io import save, output_notebook
#output_notebook()

#save(p, filename="malpractice.html", title="Medical Malpractice", resource='cdn')

#export_png(p, filename='malpractice.png')

#output_file("Map_of_malpractice_cases.html",title="Map of malpractice cases")
#p.save_figure("malpractice.png",dpi=300)
#show(p)





NameError: name 'LogColorMapper' is not defined

In [105]:
#common_states[np.where(db==2)]

In [106]:
all_slopes = []
for i in range(len(np.unique(db))):
    #print("CLUSTER "+str(i))
    #data[data["WORKSTAT"].isin(list(common_states[np.where(db==i)]))].hist(column='ORIGYEAR',bins=2018-1990,range=(1990,2018),alpha=0.5,by='WORKSTAT',figsize=(20,40),layout=(6,3))
    #plt.show()
    slope = []
    for state in list(common_states[np.where(db==i)]):
        #plt.plot(pay_count.index,pay_count[state]['count_per_10000'])
        #print(np.array(pay_count[state]['count_per_10000'].iloc[1:]))
        #print(pay_count[state]['count_per_10000'][:-1])
        slope.append(np.mean(np.array(pay_count[state]['count_per_10000'][2:])-np.array(pay_count[state]['count_per_10000'][1:-1])))
    #plt.show()
    all_slopes.append(slope)
    #print(slope)
    #print(np.mean(np.array(slope).flatten()),np.std(np.array(slope).flatten()))

#print(all_slopes[2])
#print(len(all_slopes[2]))

In [109]:
from bokeh.palettes import Category20
from bokeh.plotting import figure, output_file, show
from bokeh.models import Legend, HoverTool


p = figure(plot_width=950, plot_height=800, tools=[hover], x_axis_label='Years', y_axis_label='Malpractice cases filed per 10,000')
p.title.text = "Malpractice case per 10,000 citizens per year for a group of states which successfully diminished this number in the last two decades."


for state, color in zip(common_states[np.where(db==2)], Category20[20]):
    #print(state)
    curve = pd.DataFrame([100*len(data[(data["WORKSTAT"]==state) & (data["ORIGYEAR"]==year)])/state_pop[state_pop["State Abb"]==state][2016][0] for year in range(1990,2018)], index=range(1990,2018),columns=[state])
    #print(curve)
    plt.plot(range(1990,2018),curve,label=i,marker='.')
    p.line(curve.index, curve[state],line_width=2, color=color, alpha=1.,
           muted_color=color, muted_alpha=0.2, legend=state)


p.legend.location = "top_left"
p.legend.click_policy="mute"

from bokeh.io import save, output_notebook
output_notebook()

#output_file("interactive_legend.html", title="interactive_legend.py example")

show(p)

Loading BokehJS ...